In [ ]:
!pip -q install streamlit
!pip -q install pyngrok

In [21]:
from pyngrok import ngrok

In [38]:
%%writefile app.py

import streamlit as st
from nltk.stem.wordnet import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Model
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.cluster import KMeans
from tensorflow.keras.layers import Layer, Embedding, Input, Dense, Activation
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model,regularizers, initializers, constraints
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.backend as K
import pickle
import numpy as np
import nltk

stop = pickle.load(open("/content/drive/MyDrive/Aspect/Model_weights_17th_August/stop_words.pkl", "rb"))
lemma = pickle.load(open("/content/drive/MyDrive/Aspect/Model_weights_17th_August/lemma.pkl", "rb"))

batch_size = 512
emb_dim = 200
aspect_size = 14
epochs = 15
neg_size = 20
seed = 1234
domain = 'restaurant'
lambda_ = 0.1
emb_path = '/content/drive/MyDrive/Aspect/Model_weights_17th_August/w2v_embedding'

nltk.download('wordnet')

def preprocess(text):

  text_token = CountVectorizer().build_tokenizer()(text.lower())
  text_token = [lemma.lemmatize(i) for i in text_token if i not in stop]
  if len(text_token)>0:
    return ' '.join(text_token)+'\n'
  return '\n'  

def complete_preprocess(list_):

  processed_lst = []
  for i in list_:
    pre = preprocess(i)
    processed_lst.append(pre) 
 
  return processed_lst   

title = st.text_input('Enter review')
if title:
  test_2 = complete_preprocess([title]) 

  def split_list(list_):
    
    split_ = []
    for i in list_:
      split_.append(i.split())
    return split_   

  sp_test = split_list(test_2)

  def read_dataset( phase, vocab):

      num_hit, unk_hit, total = 0, 0, 0
      data_lst = []

      for line in phase:
          if len(line) > 0:
            indices = []
            for word in line:
                if word.replace('.','',1).isdigit():
                    indices.append(vocab['<num>'])
                    num_hit += 1
                elif word in vocab:
                    indices.append(vocab[word])
                else:
                    indices.append(vocab['<unk>'])
                    unk_hit += 1
                total += 1
            data_lst.append(indices)

      if len(phase)>20000:
        set_ = 'train'
      else:
        set_ = 'test'  
      return data_lst

  vocab_dict = pickle.load(open("/content/drive/MyDrive/Aspect/Model_weights_17th_August/vocab_dict.pkl", "rb"))
  test_sequence = read_dataset(sp_test,vocab_dict)
  test_sequence = pad_sequences(test_sequence, maxlen=156)

  class Average(Layer):
    
    def __init__(self,**kwargs):
      self.supports_masking = True
      super(Average,self).__init__(**kwargs)

    def get_config(self):
        config = super(Average, self).get_config()
        return config  

    def call(self, x, mask):
      
      mask = tf.cast(mask,tf.float32)
      mask = K.expand_dims(mask)
      x *= mask
      return tf.reduce_sum(x, axis=-2) / tf.reduce_sum(mask, axis=-2)

  class Attention(Layer):

    def __init__(self, bias=True, name=None):
      self.supports_masking = True
      self.init = initializers.get('glorot_uniform')
      self.bias = bias
      super(Attention,self).__init__(name=name)

    def get_config(self):
        config = super(Attention, self).get_config()
        config.update({"bias": self.bias})
        return config    

    def build(self, input_shape):

      self.M = self.add_weight(shape=(input_shape[0][-1], input_shape[1][-1]),initializer=self.init,name='{}_W'.format(self.name))
      if self.bias:
          self.b = self.add_weight(shape=(1,),initializer='zero',name='{}_b'.format(self.name))
      self.built=True

    def call(self, input_tensor, mask=None):
          x = input_tensor[0]
          y = input_tensor[1]
          mask = mask[0]
          tmp = tf.expand_dims(tf.matmul(y, self.M), axis=-1)
          d_i = tf.matmul(x, tmp)+self.b
          a_i = tf.nn.softmax(d_i, axis=1)
          a_i = tf.squeeze(a_i, axis=-1)
          return a_i

  def ortho_reg(weight_matrix):
      w_n = weight_matrix/tf.cast(K.epsilon()+tf.math.sqrt(tf.reduce_sum(tf.math.square(weight_matrix), axis=-1, keepdims=True)),tf.float32)
      reg = tf.reduce_sum(tf.math.square(K.dot(w_n, tf.transpose(w_n)) - tf.eye(w_n.shape[0])))
      return lambda_*reg

  class WeightedSum(Layer):
    
    def __init__(self,name=None):
      self.supports_masking = True
      super(WeightedSum,self).__init__(name=name)

    def get_config(self):
      config = super(WeightedSum, self).get_config()
      return config   

    def call(self, input_tensor, mask=None):
        x = input_tensor[0]
        a = K.expand_dims(input_tensor[1])
        weighted_input = x*a
        return tf.reduce_sum(weighted_input, axis=1)   

  class HingeLoss(Layer):

    def __init__(self,name=None):
      super(HingeLoss,self).__init__(name=name) 

    def get_config(self):
      config = super(HingeLoss, self).get_config()
      return config  

    def call(self,outputs):
      zn = outputs[0]
      zp = outputs[1]
      rs = outputs[2]
      zn = K.l2_normalize(zn, axis=-1)
      zp = K.l2_normalize(zp, axis=-1)
      rs = K.l2_normalize(rs, axis=-1)

      steps = zn.shape[1]
      pos = tf.reduce_sum(zp*rs, axis=-1, keepdims=True)
      pos = K.repeat_elements(pos, steps, axis=1)
      rs = K.expand_dims(rs, axis=-2)
      rs = K.repeat_elements(rs, steps, axis=1)
      neg = tf.reduce_sum(zn*rs, axis=-1)

      loss = tf.cast(tf.reduce_sum(K.maximum(tf.cast(0,tf.float32), (tf.cast(1,tf.float32) - pos + neg)), axis=-1, keepdims=True), tf.float32)
      return loss

  class WeightedAspectEmb(Layer):
      def __init__(self, input_dim, output_dim,init='uniform',name=None,W_regularizer=None):

          self.input_dim = input_dim
          self.output_dim = output_dim
          self.init = initializers.get(init)
          self.W_regularizer = regularizers.get(W_regularizer)
          super(WeightedAspectEmb,self).__init__(name=name)

      def get_config(self):
        config = super(WeightedAspectEmb, self).get_config()
        config.update({"input_dim": self.input_dim, 'output_dim':self.output_dim, 'init':self.init})
        return config      

      def build(self, input_shape):
        self.W = self.add_weight(shape=[self.input_dim, self.output_dim],regularizer=self.W_regularizer,initializer='uniform',name='{}_W'.format(self.name))  
        self.built = True

      def call(self, x, mask=None):
          return K.dot(x, self.W)    


  pos_input = Input(shape=(156), dtype='int32', name='sentence_input')
  neg_input = Input(shape=(neg_size, 156), dtype='int32', name='neg_input')
  embedding = Embedding(len(vocab_dict),emb_dim,  mask_zero=True, name='word_embedding', trainable=False)
  pos_emb = embedding(pos_input)
  ys = Average(name='ys')(pos_emb)
  attn = Attention(emb_dim,name='attention')([pos_emb,ys])
  zp = WeightedSum(name='zp')([pos_emb,attn])
  neg_emb = embedding(neg_input)
  zn = Average(name='zn')(neg_emb)
  pt = Dense(aspect_size, name='pt_previous')(zp)
  pt = Activation('softmax', name='pt')(pt)
  rs = Dense(emb_dim, name='asp_emb',kernel_regularizer=ortho_reg,use_bias=False)(pt)
  loss = HingeLoss(name='hinge_loss')([zn, zp, rs])

  new_model = Model(inputs=[pos_input, neg_input], outputs=loss)         
  new_model.load_weights('/content/drive/MyDrive/Aspect/Model_weights_17th_August/weights.h5')

  cluster_map_absa = {0:'Miscellaneous\n',1:'Food\n',2:'Price\n',3:'Miscellaneous\n',4:'Anecdotes\n',5:'Miscellaneous\n',6:'Food\n',
                      7:'Food\n',8:'Miscellaneous\n',9:'Miscellaneous\n',10:'Ambience\n',11:'Staff\n',12:'Food\n',13:'Food\n'}

  test_fn = Model(inputs=new_model.get_layer('sentence_input').input,outputs=new_model.get_layer('pt').output)
  aspect_probs = test_fn(test_sequence)

  label_ids = np.argsort(aspect_probs, axis=1)[:,-1]
  predict_labels_absa = [cluster_map_absa[label_id][:-1] for label_id in label_ids][0]     
  st.write('Aspect: {}'.format(predict_labels_absa))    


Overwriting app.py


In [34]:
!ngrok authtoken 1s9EWYEkYGU9J6chwq2Z0Hucw5R_2iBbA53YyhmegQY3b67nm

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [39]:
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null 

NgrokTunnel: "http://7909-35-245-178-193.ngrok.io" -> "http://localhost:80"
2021-08-26 11:47:38.198 'pattern' package not found; tag filters are not available for English
2021-08-26 11:48:02.693641: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-26 11:48:02.693695: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (3bfaf3d6f2d0): /proc/driver/nvidia/version does not exist
